In [59]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import importlib
import tweepy
import datetime
import re
import pickle
import matplotlib.pyplot as plt
from tweepy import OAuthHandler
import funcs as fs
import requests
import unidecode

Dane z wcześniejszych analiz

In [12]:
with open('geo.pickle', 'rb') as f:
    geo = pickle.load(f)
geo = geo.dropna()

#### Lista miast

In [51]:
places = pd.read_csv('dane\\places.csv', sep=';', header=None)
places.columns = ['Place_id', 'State_id', 'Place_name']
print(len(places))
print(places)

2328
      Place_id  State_id           Place_name
0            1         1          Bolesławiec
1            2         1             Gromadka
2            3         1         Nowogrodziec
3            4         1           Osiecznica
4            5         1  Warta Bolesławiecka
5            6         1                Jawor
6            7         1               Bolków
7            8         1              Męcinka
8            9         1            Mściwojów
9           10         1            Paszowice
10          11         1      Wądroże Wielkie
11          12         1              Karpacz
12          13         1               Kowary
13          14         1           Piechowice
14          15         1     Szklarska Poręba
15          16         1     Janowice Wielkie
16          17         1        Jeżów Sudecki
17          18         1          Mysłakowice
18          19         1            Podgórzyn
19          20         1      Stara Kamienica
20          21         1     

#### Usuwam polskie znaki

In [79]:
#unidecode.unidecode('ęóąśłżźćń')
places.Place_name = [unidecode.unidecode(sn).lower() for sn in list(places.Place_name)]
places.head(30)
    
#places.head(20)

,Place_id,State_id,Place_name,State_name
0,1,1,boleslawiec,dolnoslaskie
1,2,1,gromadka,dolnoslaskie
2,3,1,nowogrodziec,dolnoslaskie
3,4,1,osiecznica,dolnoslaskie
4,5,1,warta boleslawiecka,dolnoslaskie
5,6,1,jawor,dolnoslaskie
6,7,1,bolkow,dolnoslaskie
7,8,1,mecinka,dolnoslaskie
8,9,1,msciwojow,dolnoslaskie
9,10,1,paszowice,dolnoslaskie


In [10]:
states = pd.read_csv('dane\\provinces.csv', sep=';', header=None)
states.columns = ['State_id', 'State_name']
print(len(states))
print(states)

16
    State_id           State_name
0          1         dolnośląskie
1          2   kujawsko-pomorskie
2          3            lubelskie
3          4             lubuskie
4          5              łódzkie
5          6          małopolskie
6          7          mazowieckie
7          8             opolskie
8          9         podkarpackie
9         10            podlaskie
10        11            pomorskie
11        12              śląskie
12        13       świętokrzyskie
13        14  warmińsko-mazurskie
14        15        wielkopolskie
15        16   zachodniopomorskie


In [63]:
states.State_name = [unidecode.unidecode(sn) for sn in list(states.State_name)]
states

,State_id,State_name
0,1,dolnoslaskie
1,2,kujawsko-pomorskie
2,3,lubelskie
3,4,lubuskie
4,5,lodzkie
5,6,malopolskie
6,7,mazowieckie
7,8,opolskie
8,9,podkarpackie
9,10,podlaskie


#### Dodanie nazwy wojewodztwa

In [74]:
places = pd.merge(places, states, how='left', on='State_id')

In [93]:
geo.tail(20)

,tw_text,location,user_id,user_name,fav_cnt,friends_cnt,foll_cnt,state,new_loc
9943,"I do każdej strony mam inne dane do logowania,...","New York, USA",1966934958,fabulous bitch,1308,373,312,New York,
9950,@Duda1967Adam @goltarr @lapszynski @mswierczyn...,"Warszawa, Polska",2370861218,MariuszMarszałkowski,3526,275,278,mazowieckie,
9952,@SOlechnowicz I cyk selfiaczek z Giertychem.,"Białystok, Polska",470556736,Tomasz Więsław,1982,450,54,podlaskie,
9953,Każda niezaplanowana miłość\njest prawdziwa*,"Warszawa, Polska",1044550715270672384,rosse📌,28,611,398,mazowieckie,
9954,ICYMI... https://t.co/vTPjMSCFZH,Bristol,335174279,Daniel Evans,3547,1512,2006,England,
9958,Paweł Wójcik z Polsatu Sport do kibiców na waż...,"Warszawa, Polska",2393089614,Marcin Piechota,3681,376,1219,mazowieckie,
9963,"@wierchowienski @CinemaCityPL Wiem, ale dyskus...","Warszawa, Polska",3973882731,Pennywise,24363,450,533,mazowieckie,
9964,@PARADlSEJEON o nie ahahhaah,neverland//taejin #1enthusiast,2597566308,❁ωιѕzι миιαм❁STREAM WIND FLOWER🌈,41648,676,1143,California,
9969,jakich owocow nie lubisz? — praktycznie wszyst...,pjm knj lcr jk kb,736594059356168192,spotlight,35724,749,983,Provincia Puntarenas,
9973,Podobam się mojemu koledze Shit,"Kraków, Polska",3871987293,julinka,3192,142,544,małopolskie,


In [13]:
print('Liczba tweetów ', len(geo))


Liczba tweetów  4214


#### Przetwarzanie tekstu w kolumnie "location"

In [183]:
# 1
# województwo pojawia się w polu
# 2
# usuwam polskie znaki
# 3
# miasta ==
# 4
# poprawa Warsaw
# 5
# dzielenie najpierw po przecinku, później po spacji

def get_state_fast(location):
    location = unidecode.unidecode(re.sub(r'[\\/;\:]', ',', location).lower())
    location=re.sub('polska', ' ', location)
    location=re.sub('poland', ' ', location)
    location=re.sub('warsaw', 'warszawa', location).strip()
    #print(location)

    loc_parts = location.split(',')
    try:
        s = [part for part in loc_parts if part in list(states.State_name)][0]
    except:
        s= ''
        
    if len(s)==0:
        for part in loc_parts:
            if part in list(places.Place_name):
                s = places.State_name.loc[places.Place_name==part].values[0]
                break
            else:
                location = unidecode.unidecode(re.sub(r'[,-]', ' ', location).lower())
                loc_parts = location.split(' ')
                for part in loc_parts:
                    if part.strip() in list(places.Place_name):
                        s = places.State_name.loc[places.Place_name==part.strip()].values[0]
                        break
    
    return s

In [184]:
get_state_fast('Lublin-Poland')

'lubelskie'

In [185]:
geo['new_loc'] =  [get_state_fast(l) for l in geo.location]

In [186]:
print(len(geo), len(geo[geo.new_loc!='']))
#4214 201
# usuwam polskie znaki
#4214 201
#miasta
#4214 1526
#korekta na Warsaw
#4214 1574
# inaczej dzielenie
#4214 1614

4214 1614


In [153]:
"""Żędowice
Wielkopolska
Warsaw
Licheń"""
'  asd  ff  '.strip()
#[p for p in list(places.Place_name) if p.startswith('warsz')]
#list(places.Place_name).startswith('l')
#re.search(r'^l','lichen')

'asd  ff'

In [173]:
geo[geo.new_loc==''][700:750]

,tw_text,location,user_id,user_name,fav_cnt,friends_cnt,foll_cnt,state,new_loc
2469,to co się dzieje w moim życiu obecnie jest tak...,대한민국 청주시,1063069957829939201,Victoria,1159,142,19,충북,
2470,@oparyrteci chora jestes n o cb chodzi,"➳suju, bts, day6, monsta x, jbj, tøp, mcr, bmt...",748070270955556864,emø pasøżyt °,98054,712,2064,England,
2476,@uskrzydlenie dobranoc,piekło,942758764482002944,𝒥,58883,172,1319,warmińsko-mazurskie,
2477,"""Wybory za nami, ale w Łódzkiem walka o marsza...",w przestrzeni czasu,2962132799,Bogumiła Mirosławska,30002,2938,3763,Western,
2478,ogólnie chyba przestanę się malować do szkoły ...,Neverland,948584865473204225,🌿olusia(◜௰◝) 結,54749,1201,458,California,
2480,"@njhflicker_ Ee, zwykłym moja koleżanka robiła...",Po(LA)nd,549030562,ＡＬＩＣＪＡ 👑,1620,4251,2696,North Dakota,
2495,"jest 23, a ja właśnie zobaczyłam na tl tego ch...",Godric's Hollow,622828487,atena,63721,215,1544,England,
2498,kocham jak mój tata przełącza kanały i usypia ...,gryffindor,946129216483053568,romantic works,5682,162,96,Wisconsin,
2499,"ale pewnie, przecież my nikogo nie zachęcamy d...",S.T.A.R. Labs,958734414283976704,czaruś 🌼❄,3666,138,307,Salzburg,
2500,@happychimmie ULTNIJ NATYCHMIAST U GO GURL,kdy lty ldh lnk,1000685516415799296,ً ¹²⁷,27644,1116,1193,Scotland,
